In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import os
import zipfile
import shutil

In [2]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "download.default_directory": r"D:\Program Files (x86)\百度云同步盘\Dropbox\-E·J- 2014.5.1\2016.12.15 店小秘数据分析\2017.10.24 各渠道运费明细\燕文\\", # IMPORTANT - ENDING SLASH V IMPORTANT
         "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

In [3]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [4]:
browser.get('http://portal.yw56.com.cn/login')

In [5]:
yw_account = 401614
yw_psw = 'enzo123456'

In [6]:
def find_elements_by_css_selector(class_name, order_num):
    class_name = class_name.replace(" ", ".")
    class_name = '.' + class_name
    return browser.find_elements_by_css_selector(class_name)[order_num]

In [7]:
time.sleep(5)
yw_account_input = find_elements_by_css_selector('ant-input ant-input-lg', 0)
yw_psw_input = find_elements_by_css_selector('ant-input ant-input-lg', 1)

In [8]:
# 输入账户密码登录
yw_account_input.click()
yw_account_input.send_keys(yw_account)
yw_psw_input.click()
yw_psw_input.send_keys(yw_psw)
time.sleep(10)

In [9]:
# 登录
# login_button = find_elements_by_css_selector("ant-btn antd-pro-components-login-index-submit ant-btn-primary ant-btn-lg", 0)
# login_button.click()
# time.sleep(10)

In [10]:
# 点击我的历史账单
browser.get('https://portal.yw56.com.cn/myBill/bill/historyBill')
time.sleep(5)

In [11]:
# 选择‘账单’
# find_elements_by_css_selector('ant-select ant-select-enabled', 1).click()
# time.sleep(5)
# browser.find_element_by_xpath("//ul[@class='ant-select-dropdown-menu  ant-select-dropdown-menu-root ant-select-dropdown-menu-vertical']/li[text()='账单']").click()

In [12]:
# 选择‘账单’
# https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value
# browser.find_element_by_xpath("//select[@name='transType']/option[text()='账单']").click()

In [13]:
# 日期选择器
import datetime as dt
from datetime import timedelta
today_date = dt.datetime.today()
ninety_days_ago = today_date - timedelta(days=120)
ninety_days_ago_str_format = ninety_days_ago.strftime('%Y-%m-%d')
today_date_str_format = today_date.strftime('%Y-%m-%d')
# ninety_days_ago_str_format

In [14]:
# 设置日期
# find_elements_by_css_selector('ant-calendar-picker-input ant-input', 0).click()
# time.sleep(5)

In [15]:
# 选择结束日期
# find_elements_by_css_selector('ant-calendar-cell ant-calendar-last-day-of-month', -1).click()

In [16]:
# 翻上一页
# find_elements_by_css_selector('ant-calendar-prev-month-btn', 0).click()
# find_elements_by_css_selector('ant-calendar-prev-month-btn', 0).click()

In [17]:
# 选择起始日期
# find_elements_by_css_selector('ant-calendar-cell ant-calendar-last-month-cell', 0).click()

In [18]:
# find_elements_by_css_selector('ant-calendar-picker', 0).click()
# start_date = find_elements_by_css_selector('ant-calendar-range-picker-input', 0)
# start_date.click()
# time.sleep(2)
# start_date.clear()
# time.sleep(5)
# start_date.send_keys(ninety_days_ago_str_format)
# start_date.setAttribute("value", ninety_days_ago_str_format)
# browser.execute_script("arguments[0].setAttribute('value', '" + ninety_days_ago_str_format + "')", start_date);

# end_date = find_elements_by_css_selector('ant-calendar-range-picker-input', 1)

# browser.execute_script("arguments[0].setAttribute('value', '" + today_date_str_format + "')", end_date);

In [19]:
# 点击查询
# search_bill = find_elements_by_css_selector("ant-btn ant-btn-primary", 0)
# search_bill.click()
# time.sleep(10)

In [20]:
# # 设置条目数
# num_of_rows = find_elements_by_css_selector('btn-group dropup', 0)
# num_of_rows.click()

# # 点击最大条目数
# # https://stackoverflow.com/questions/34583524/python-selenium-click-on-all-li-in-ul
# # https://stackoverflow.com/questions/18026766/selenium-select-element-with-multiple-attributes
# num_of_max_rows = browser.find_elements_by_xpath("//ul[@class = 'dropdown-menu'][@role = 'menu']//li[not(@class)]")
# # num_of_max_rows[-1].text
# num_of_max_rows[-1].click()
# time.sleep(5)

In [21]:
# # 查看有多少条记录
# # https://stackoverflow.com/questions/14831041/how-to-count-no-of-rows-in-table-from-web-application-using-selenium-python-webd
# row_count = len(browser.find_elements_by_xpath("//table[@id='table']/tbody/tr"))
# print(row_count)
# time.sleep(5)

In [22]:
# # 点击每条账单
# '''
# bill_id = 0
# bill_location = browser.find_elements_by_xpath("//table[@id='table']/tbody/tr[@data-index='" + str(bill_id) + "']/td")[5]
# bill_html = "http://portal.yw56.com.cn" + bill_location.get_attribute('innerHTML').split('"')[1].replace('amp;','')
# browser.get(bill_html)
# '''

In [23]:
# # 点击下载
# '''
# bill_download = find_elements_by_css_selector("btn", 1)
# bill_download.click()
# '''

In [24]:
# 逐条点击
item_in_page = len(browser.find_elements_by_css_selector('.ant-table-row.ant-table-row-level-0'))
# browser.find_element_by_xpath("//tbody[@class='ant-table-tbody']/tr[@data-row-key='0']/td[6]").click()
# time.sleep(5)

In [ ]:
# 点击下载
# find_elements_by_css_selector('ant-btn ant-btn-primary', 1).click()
# time.sleep(5)

In [ ]:
# 逐个下载
i = 1
fanye = 0
try:
    while i <= item_in_page:
        get_result = True
        while get_result == True:
            try:
                print('there are {} item in this page'.format(item_in_page))
                # 回到账单页
                browser.get('https://portal.yw56.com.cn/myBill/bill/historyBill')
                time.sleep(5)
                # 选择‘账单’
                find_elements_by_css_selector('ant-select ant-select-enabled', 1).click()
                time.sleep(5)
                browser.find_element_by_xpath("//ul[@class='ant-select-dropdown-menu  ant-select-dropdown-menu-root ant-select-dropdown-menu-vertical']/li[text()='账单']").click()
                time.sleep(5)
                # 设置日期
                find_elements_by_css_selector('ant-calendar-picker-input ant-input', 0).click()
                time.sleep(5)
                # 选择结束日期
                find_elements_by_css_selector('ant-calendar-cell ant-calendar-today ant-calendar-selected-end-date ant-calendar-selected-day', 0).click()    
                time.sleep(5)
                # 翻上一页
                find_elements_by_css_selector('ant-calendar-prev-month-btn', 0).click()
                time.sleep(5)
                find_elements_by_css_selector('ant-calendar-prev-month-btn', 0).click()
                time.sleep(5)
                # 选择起始日期
                find_elements_by_css_selector('ant-calendar-cell ant-calendar-last-day-of-month', 0).click()
                time.sleep(5)
                # 点击查询
                search_bill = find_elements_by_css_selector("ant-btn ant-btn-primary", 0)
                search_bill.click()
                time.sleep(5)

                # 观察是否需要翻页
                if fanye > 0:
                    for w in range(fanye):
                        find_elements_by_css_selector('ant-pagination-next', 0).click()
                        print('翻页')
                        time.sleep(5)

                # 去到页面底部
                body = browser.find_element_by_css_selector('body')
                body.send_keys(Keys.END)
                time.sleep(5)
                if i <= 5:
                    body.send_keys(Keys.CONTROL + Keys.HOME)
                    time.sleep(5)

                # 本页有多少结果
                item_in_page = len(browser.find_elements_by_css_selector('.ant-table-row.ant-table-row-level-0'))
                print('{} item in page'.format(item_in_page))                        
                        
                # 点击账单金额
                try:
                    browser.find_element_by_xpath("//tbody[@class='ant-table-tbody']/tr[@data-row-key='" + str(i - 1) + "']/td[7]").click()
                    print('点击了账单金额')
                    time.sleep(5)
                    # 点击下载
                    find_elements_by_css_selector('ant-btn ant-btn-primary', 1).click()
                    print('点击了账单下载')
                    time.sleep(5)
                except Exception as err:
                    print('Exception occurred: ' + str(err))

                # 再点击下一个账单
                i = i + 1

                # 如果 i > 10，则点击下一页
                if i > 10:
                    find_elements_by_css_selector('.ant-pagination-next', 0).click()
                    time.sleep(5)
                    item_in_page = len(browser.find_elements_by_css_selector('.ant-table-row.ant-table-row-level-0'))
                    i = 1
                    fanye = fanye + 1
                    
                get_result = False
            except Exception as err:
                print('Exception occurred: ' + str(err))
                get_result = True

except Exception as err:
    print('Exception occurred: ' + str(err))

there are 6 item in this page
Exception occurred: Message: element not interactable
  (Session info: chrome=100.0.4896.127)

there are 6 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
there are 10 item in this page
10 item in page
点击了账单金额
Exception occurred: Message: element click intercepted: Element <button type="button" class="ant-btn ant-btn-primary" style="margin-left: 10px;">...</button> is not clickable at point (1692, 66). Other element would receive the click: <div class="notice-box-tit">...</div>
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
点击了账单金额
点击了账单下载
the

10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='28']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='29']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='30']/td[7]"}
  (Session info: chrome=100.0.4896.127)



10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='50']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='51']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='52']/td[7]"}
  (Session info: chrome=100.0.4896.127)



10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='72']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='73']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='74']/td[7]"}
  (Session info: chrome=100.0.4896.127)



10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='94']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='95']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='96']/td[7]"}
  (Session info: chrome=100.0.4896.127)



10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='116']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='117']/td[7]"}
  (Session info: chrome=100.0.4896.127)

Exception occurred: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=100.0.4896.127)

there are 10 item in this page
10 item in page
Exception occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tbody[@class='ant-table-tbody']/tr[@data-row-key='118']/td[7]"}
  (Session info: chrome=100.0.4896.127

In [ ]:
# browser.find_element_by_xpath("//tbody[@class='ant-table-tbody']/tr[@data-row-key='" + str(i - 1) + "']/td[7]").text

In [ ]:
# for i in range(row_count):
#     # 点击我的历史账单
#     browser.get('http://portal.yw56.com.cn/bill/index?type=1')
#     # 选择‘账单’
#     # https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value
#     browser.find_element_by_xpath("//select[@name='transType']/option[text()='账单']").click()
#     # 日期选择器
#     today_date = dt.datetime.today()
#     ninety_days_ago = today_date - timedelta(days=90)
#     ninety_days_ago_str_format = ninety_days_ago.strftime('%Y-%m-%d')
#     # ninety_days_ago_str_format
#     # 设置日期
#     start_date = find_elements_by_css_selector('form-control', 0)
#     start_date.clear()
#     start_date.send_keys(ninety_days_ago_str_format)
#     # 点击查询
#     search_bill = find_elements_by_css_selector("btn", 0)
#     search_bill.click()
#     time.sleep(10)
#     # 设置条目数
#     num_of_rows = find_elements_by_css_selector('btn-group dropup', 0)
#     num_of_rows.click()
#     # 点击最大条目数
#     # https://stackoverflow.com/questions/34583524/python-selenium-click-on-all-li-in-ul
#     # https://stackoverflow.com/questions/18026766/selenium-select-element-with-multiple-attributes
#     num_of_max_rows = browser.find_elements_by_xpath("//ul[@class = 'dropdown-menu'][@role = 'menu']//li[not(@class)]")
#     # num_of_max_rows[-1].text
#     num_of_max_rows[-1].click()
#     time.sleep(5)
#     # 点击每条账单
#     bill_id = i
#     try:
#         bill_location = browser.find_elements_by_xpath("//table[@id='table']/tbody/tr[@data-index='" + str(bill_id) + "']/td")[5]
#         print(bill_location)
#         bill_html = "http://portal.yw56.com.cn" + bill_location.get_attribute('innerHTML').split('"')[1].replace('amp;','')
#         print(bill_html)
#         browser.get(bill_html)
#         # 点击下载
#         bill_download = find_elements_by_css_selector("btn", 1)
#         bill_download.click()
#         time.sleep(5)
#     except Exception as err:
#         print('Exception occurred: ' + str(err))